# CartoonGAN 논문의 모델 구조
<img src="https://aruie.github.io/assets/post/191114-01.png" width=700>


# generator - 방법 2 사용!
- 초기화 단계 : 10 epoch만큼 먼저 훈련

## input
- resizing된 사진 (resizing 함수를 이용하여 300x300으로 미리 resizing)

In [ ]:
# Google Drive mount
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 참고 : https://www.tensorflow.org/api_docs

In [ ]:
# 사진 있는 디렉토리 위치
photo_path = '/content/drive/MyDrive/GoogleColab/CartoonGAN_Project/data/photo_data'

# 사진 파일명 리스트 저장
photoName_list = os.listdir(photo_path)
print(photoName_list[:10])  # 앞의 10개만 파일명 보기
print(len(photoName_list))  # 사진 총 몇 장 있는 지 보기

['50817142731_dfc154ea75_o.jpg', '50819927577_2286b936e1_k.jpg', '50817727238_b7f3f5acd9_o.jpg', '50821206327_ea91c5e259_o.jpg', '50820323176_60016143b1_o.jpg', '40882481714_50b8a7490e_o.jpg', '44926777104_4ed4b51861_o.jpg', '27585811288_40d99dc08e_o.jpg', '5508784414_3ec3992ddc_o.jpg', '25690386427_7ef979d9ab_k.jpg']
43


## 모델링

In [ ]:
# 네트워크를 만들 땐
# 1. sequential 모델을 사용한 네트워크
# 2. 함수형 API를 사용한 네트워크

# 2가지 방법으로 만들 수 있다.

# 1번 방법 ->  일렬로 층을 쌓은 네트워크를 빠르게 만들 때 사용하기 좋다.
# 2번 방법 -> 한 층의 출력이 여러 개의 별도의 층으로 전달되는 네트워크를 만들 때 유연성있게 만들기 좋다.
#          -> 심층 신경망 구조를 설계하는데 자유롭다.

In [ ]:
# residual block은 똑같은 구조가 generator안에 8번 반복되므로 함수로 따로 만들자.
def generator_residual_block(x):
  shortcut = x
  x = Conv2D(kernel_size = 3,
            filters = 256,
            strides = 1,
            padding = "same"
            )(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(kernel_size = 3,
            filters = 256,
            strides = 1,
            padding = "same")(x)
  x = BatchNormalization()(x)
  x = layers.Add()([x, shortcut])  # identity shortcut connection  # elementwise sum
  
  return x

up-convolution 영역
1. UpSampling2D + Conv2D
2. Conv2DTranspose
- 두 가지 방법 다 사용해보고 어떤 것이 좋은 결과가 나오는 지 직접 확인해보고 결정해야함


- UpSampling2D docs 참고 : https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D
- Conv2DTranspose docs 참고 : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose



## 방법 1 - UpSampling2D + Conv2D 이용

In [ ]:
# 우선 input에 사진 이미지를 넣지 않고 숫자만 넣고
# 틀만 짜보자.

input_shape1 = (300, 300, 3)

input_layer1 = Input(shape = input_shape1)  # resizing된 사진 넣기

# flat-convolution 영역
net1 = Conv2D(kernel_size = 7,
              filters = 64,
              strides = 1,
              padding = "same"
              )(input_layer1)
net1 = BatchNormalization()(net1)
net1 = ReLU()(net1)

# down-convolution 영역
net1 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 2,
              padding = "same"
              )(net1)

net1 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 1,
              padding = "same"
              )(net1)
net1 = BatchNormalization()(net1)
net1 = ReLU()(net1)

net1 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 2,
              padding = "same"
              )(net1)
net1 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 1,
              padding = "same"
              )(net1)
net1 = BatchNormalization()(net1)
net1 = ReLU()(net1)

# 8 residual block 영역
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)
net1 = generator_residual_block(net1)

# up-convolution 영역
net1 = UpSampling2D(size = (2, 2)
                    )(net1)
net1 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 1,
              padding = "same"
              )(net1)
net1 = BatchNormalization()(net1)
net1 = ReLU()(net1)

net1 = UpSampling2D(size = (2, 2)
                    )(net1)
net1 = Conv2D(kernel_size = 3,
              filters = 64,
              strides = 1,
              padding = "same"
              )(net1)
net1 = BatchNormalization()(net1)
net1 = LeakyReLU()(net1)

# output layer 영역
net1 = Conv2D(kernel_size = 7,
              filters = 3,
              strides = 1,
              padding = "same"
              )(net1)

In [ ]:
model1 = Model(inputs = input_layer1,
               outputs = net1,
               name = "generator1")
model1.summary()

Model: "generator1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d_614 (Conv2D)             (None, 300, 300, 64) 9472        input_31[0][0]                   
__________________________________________________________________________________________________
batch_normalization_519 (BatchN (None, 300, 300, 64) 256         conv2d_614[0][0]                 
__________________________________________________________________________________________________
re_lu_287 (ReLU)                (None, 300, 300, 64) 0           batch_normalization_519[0][0]    
_________________________________________________________________________________________

In [ ]:
# 출력 shape : (None, 300, 300, 3)
# 300x300 의 컬러 이미지로 잘 나오는 것 확인.

## 방법 2 - Conv2DTranspose 이용

In [ ]:
# 우선 input에 사진 이미지를 넣지 않고 숫자만 넣고
# 틀만 짜보자.

input_shape2 = (300, 300, 3)

input_layer2 = Input(shape = input_shape2)  # resizing된 사진 넣기

# flat-convolution 영역
net2 = Conv2D(kernel_size = 7,
              filters = 64,
              strides = 1,
              padding = "same"
              )(input_layer2)
net2 = BatchNormalization()(net2)
net2 = ReLU()(net2)

# down-convolution 영역
net2 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 2,
              padding = "same"
              )(net2)

net2 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 1,
              padding = "same"
              )(net2)
net2 = BatchNormalization()(net2)
net2 = ReLU()(net2)

net2 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 2,
              padding = "same"
              )(net2)
net2 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 1,
              padding = "same"
              )(net2)
net2 = BatchNormalization()(net2)
net2 = ReLU()(net2)

# 8 residual block 영역
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)
net2 = generator_residual_block(net2)

# up-convolution 영역
net2 = Conv2DTranspose(kernel_size = 3,
                       filters = 128,
                       strides = (2, 2),
                       padding = "same"
                       )(net2)
net2 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 1,
              padding = "same"
              )(net2)
net2 = BatchNormalization()(net2)
net2 = ReLU()(net2)

net2 = Conv2DTranspose(kernel_size = 3,
                       filters = 64,
                       strides = (2, 2),
                       padding = "same"
                       )(net2)
net2 = Conv2D(kernel_size = 3,
              filters = 64,
              strides = 1,
              padding = "same"
              )(net2)
net2 = BatchNormalization()(net2)
net2 = LeakyReLU()(net2)

# output layer 영역
net2 = Conv2D(kernel_size = 7,
              filters = 3,
              strides = 1,
              padding = "same"
              )(net2)

In [ ]:
model2 = Model(inputs = input_layer2,
              outputs = net2,
              name = "generator2")
model2.summary()

Model: "generator2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d_638 (Conv2D)             (None, 300, 300, 64) 9472        input_32[0][0]                   
__________________________________________________________________________________________________
batch_normalization_540 (BatchN (None, 300, 300, 64) 256         conv2d_638[0][0]                 
__________________________________________________________________________________________________
re_lu_299 (ReLU)                (None, 300, 300, 64) 0           batch_normalization_540[0][0]    
_________________________________________________________________________________________

In [ ]:
# 출력 shape : (None, 300, 300, 3)
# 300x300 의 컬러 이미지로 잘 나오는 것 확인.

그렇다면 방법 1 - UpSampling2D + Conv2D를 사용하는 방법과

방법 2 - Conv2DTranspose를 사용하는 방법 중 어떤 것을 사용해야 적절할까?

<img src="https://aruie.github.io/assets/post/191114-01.png" width=700>

이와 같이 논문에서 나오는 모델의 구조를 보면

kernel_size, filters, strides가 다 디테일하게 나와있다.

그러므로 kernel_size, filters에 대해서는 파라미터 조정을 해줄 수 없는 방법 1의 UpSampling2D를 사용하는 것보다

kernel_size, filters, strides에 대해 파라미터 조정을 다 해줄 수 있는 방법 2의 Conv2DTranspose를 사용하는 방법이 더 적절해보인다!

## 방법 1, 방법 2 중 어떤 것 사용? -> 방법 2 사용

## compile

In [ ]:
# generator 컴파일
model2.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# discriminator

## input
- 사진
- 엣지 smoothing된 만화 이미지
- 카툰화된 사진

## 모델링

In [ ]:
# 우선 input에 사진 이미지를 넣지 않고 숫자만 넣고
# 틀만 짜보자.
input_shape3 = (300, 300, 3)
input_layer3 = Input(input_shape3)  # 사진, 엣지 smoothing된 만화 이미지, 카툰화된 사진이 들어옴

net3 = Conv2D(kernel_size = 3,
              filters = 32,
              strides = 1,
              padding = "same"
              )(input_layer3)
net3 = LeakyReLU(alpha=0.2)(net3)

net3 = Conv2D(kernel_size = 3,
              filters = 64,
              strides = 2,
              padding = "same"
              )(net3)
net3 = LeakyReLU(alpha=0.2)(net3)
net3 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 1,
              padding = "same"
              )(net3)
net3 = BatchNormalization()(net3)
net3 = LeakyReLU(alpha=0.2)(net3)

net3 = Conv2D(kernel_size = 3,
              filters = 128,
              strides = 2,
              padding = "same"
              )(net3)
net3 = LeakyReLU(alpha=0.2)(net3)
net3 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 1,
              padding = "same"
              )(net3)
net3 = BatchNormalization()(net3)
net3 = LeakyReLU(alpha=0.2)(net3)

net3 = Conv2D(kernel_size = 3,
              filters = 256,
              strides = 1,
              padding = "same"
              )(net3)
net3 = BatchNormalization()(net3)
net3 = LeakyReLU(alpha=0.2)(net3)

net3 = Conv2D(kernel_size = 3,
              filters = 1,
              strides = 1,
              padding = "same"
              )(net3)

In [ ]:
model3 = Model(inputs = input_layer3,
              outputs = net3,
              name = "discriminator")
model3.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d_662 (Conv2D)          (None, 300, 300, 32)      896       
_________________________________________________________________
leaky_re_lu_60 (LeakyReLU)   (None, 300, 300, 32)      0         
_________________________________________________________________
conv2d_663 (Conv2D)          (None, 150, 150, 64)      18496     
_________________________________________________________________
leaky_re_lu_61 (LeakyReLU)   (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_664 (Conv2D)          (None, 150, 150, 128)     73856     
_________________________________________________________________
batch_normalization_561 (Bat (None, 150, 150, 128)   

In [ ]:
# 출력 shape : (None, 75, 75, 1)

# compile
model3.compile(loss = loss, optimizer = optimizer, metrics = metrics)

# generator_web
- 위의 generator를 훈련 데이터로 학습을 시킨 후, 그 가중치를 피클로 저장하여 generator_web만 서버에 올린다!